In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00


In [12]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def preprocess_text(text, tokenizer):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    return inputs

def train_model(model, train_inputs, train_labels, num_epochs=3, batch_size=16):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    model.to(device)

    train_dataset = torch.utils.data.TensorDataset(train_inputs['input_ids'],
                                                  train_inputs['attention_mask'],
                                                  train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f}')

def classify_sentence(model, sentence, tokenizer):
    model.eval()
    input_data = preprocess_text(sentence, tokenizer)
    input_ids = input_data['input_ids'].to(device)
    attention_mask = input_data['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    return predicted_label

if __name__ == '__main__':
    # Load pre-trained BERT model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Example training data (paragraphs)
    train_paragraphs = [
        "Yoga is an ancient practice that originated in India thousands of years ago. It encompasses a holistic approach to physical and mental well-being, combining physical postures (asanas), breathing exercises (pranayama), and meditation techniques. The practice of yoga aims to harmonize the body, mind, and spirit, promoting balance, flexibility, strength, and inner peace. Through the regular practice of yoga, individuals can experience a myriad of benefits. Physically, yoga helps improve flexibility, increase muscle tone and strength, and enhance overall body awareness. It can also alleviate chronic pain, reduce the risk of injury, and improve posture and alignment.Mentally and emotionally, yoga provides a space for mindfulness and self-reflection. It can reduce stress, anxiety, and depression by promoting relaxation and calming the mind. Regular yoga practice cultivates focus, concentration, and mental clarity. It also encourages self-acceptance, self-compassion, and a deeper connection with oneself and others. Furthermore, yoga offers a spiritual dimension, allowing practitioners to explore and deepen their spiritual connection. It encourages the development of inner wisdom, compassion, and gratitude. Yoga philosophy encompasses principles such as non-violence (ahimsa), truthfulness (satya), contentment (santosha), and self-discipline (tapas). Yoga is a versatile practice that offers various styles and approaches to suit individual preferences and needs. Hatha yoga focuses on physical postures and breath control, while Vinyasa yoga emphasizes flowing movements coordinated with breath. Other styles like Ashtanga, Kundalini, and Iyengar each have their unique characteristics and benefits. Practicing yoga is accessible to people of all ages and fitness levels. It can be adapted to accommodate different body types, abilities, and limitations. Whether practiced in a group setting or at home, yoga provides an opportunity to nurture oneself, find inner balance, and cultivate overall well-being.",
        "Football, also known as soccer, is a popular sport played worldwide. It involves two teams competing against each other to score goals by kicking a ball into the opposing team's net. Football requires physical agility, speed, endurance, and teamwork. The game of football consists of various positions, including forwards, midfielders, defenders, and goalkeepers. Each position has its unique role and responsibilities. The forwards primarily focus on scoring goals, while midfielders play a critical role in controlling the game and facilitating transitions between defense and attack. Defenders aim to prevent the opposing team from scoring, and goalkeepers protect the goal from incoming shots. Football matches are played on a rectangular field with two goals at opposite ends. The objective is to move the ball across the field using various passing, dribbling, and shooting techniques. Strategies and tactics are employed to outwit the opposing team and create scoring opportunities.Beyond the physical aspects, football fosters camaraderie, sportsmanship, and teamwork among players. It promotes discipline, dedication, and perseverance. Football matches often evoke a sense of passion, excitement, and unity among fans and supporters. The popularity of football has led to the establishment of professional leagues and international tournaments, such as the FIFA World Cup and UEFA Champions League. These events showcase the highest level of competition and bring people from different nations together through their shared love for the sport. Football enthusiasts engage in watching matches, playing in local teams, and participating in fantasy football leagues. The sport has a significant cultural impact and serves as a source of entertainment, social connection, and inspiration for millions of people worldwide. Basketball is a popular sport played around the world. It involves two teams competing to score points by shooting a ball into the opposing team's basket. While basketball requires physical fitness and coordination, it is not directly related to yoga."
    ]
    train_labels = [1, 0]  # 1 for yoga-related, 0 for not yoga-related

    # Example test sentence
    test_sentence = "Play any video for relaxation mediatation"

    # Preprocess and train the model
    train_inputs = preprocess_text(train_paragraphs, tokenizer)
    train_labels = torch.tensor(train_labels)
    train_model(model, train_inputs, train_labels)

    # Perform inference on the test sentence
    predicted_label = classify_sentence(model, test_sentence, tokenizer)
    print(predicted_label)  # 1 (yoga-related)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1/3 - Loss: 0.5409
Epoch 2/3 - Loss: 0.6999
Epoch 3/3 - Loss: 0.6676
